# Heart Disease Prediction - EDA & Imputation Analysis

This notebook performs exploratory data analysis using reusable plotting functions.

## Table of Contents
1. [Setup & Data Loading](#section1)
2. [Missing Values Analysis](#section2)
3. [Target Variable Analysis](#section3)
4. [Numerical Features Analysis](#section4)
5. [Categorical Features Analysis](#section5)
6. [Correlation Analysis](#section6)
7. [Outlier Detection](#section7)
8. [Statistical Tests](#section8)

---
## 1. Setup & Data Loading

In [24]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join('..')))

import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import chi2_contingency

import src as plots

train_raw = pd.read_csv('../data/raw/train.csv')
test_raw = pd.read_csv('../data/raw/test.csv')

print(f"Training set shape: {train_raw.shape}")
print(f"Test set shape: {test_raw.shape}")
print("\nFirst few rows:")
display(train_raw.head())

Training set shape: (732, 14)
Test set shape: (184, 13)

First few rows:


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label
0,51.0,1.0,1.0,125.0,213.0,0.0,2.0,125.0,1.0,1.4,1.0,1.0,3.0,0
1,54.0,1.0,3.0,120.0,237.0,0.0,0.0,150.0,1.0,1.5,-9.0,-9.0,7.0,2
2,63.0,1.0,4.0,140,0,?,2.0,149,0,2,1,?,?,2
3,52.0,0.0,2.0,140.0,-9.0,0.0,0.0,140.0,0.0,0.0,-9.0,-9.0,-9.0,0
4,55.0,1.0,4.0,140.0,217.0,0.0,0.0,111.0,1.0,5.6,3.0,0.0,7.0,3


In [25]:
print("Dataset Info:")
print(train_raw.info())
print("\nBasic Statistics:")
display(train_raw.describe())
display(train_raw['ca'].unique())

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       732 non-null    float64
 1   sex       732 non-null    float64
 2   cp        732 non-null    float64
 3   trestbps  732 non-null    object 
 4   chol      732 non-null    object 
 5   fbs       732 non-null    object 
 6   restecg   732 non-null    float64
 7   thalach   732 non-null    object 
 8   exang     732 non-null    object 
 9   oldpeak   732 non-null    object 
 10  slope     732 non-null    object 
 11  ca        732 non-null    object 
 12  thal      732 non-null    object 
 13  label     732 non-null    int64  
dtypes: float64(4), int64(1), object(9)
memory usage: 80.2+ KB
None

Basic Statistics:


,age,sex,cp,restecg,label
count,732.000000,732.000000,732.000000,732.000000,732.000000
mean,53.364754,0.789617,3.250000,0.602459,1.132514
std,9.306868,0.407859,0.923363,0.802966,1.257615
min,28.000000,0.000000,1.000000,0.000000,0.000000
25%,47.000000,1.000000,3.000000,0.000000,0.000000
50%,54.000000,1.000000,4.000000,0.000000,1.000000
75%,60.000000,1.000000,4.000000,1.000000,2.000000
max,76.000000,1.000000,4.000000,2.000000,4.000000


array(['1.0', '-9.0', '?', '0.0', '2.0', '1', '3.0', '0', '2'],
      dtype=object)

### Data Cleaning
Convert '?' to NaN and '-9' values to NaN for proper missing value handling.

In [26]:
train_clean = train_raw.replace('?', np.nan)
test_clean = test_raw.replace('?', np.nan)

for col in train_clean.columns:
    if col != 'label':
        train_clean[col] = pd.to_numeric(train_clean[col], errors='coerce')
        if col in test_clean.columns:
            test_clean[col] = pd.to_numeric(test_clean[col], errors='coerce')

train_clean = train_clean.replace(-9.0, np.nan)
test_clean = test_clean.replace(-9.0, np.nan)

print(f"Cleaned training shape: {train_clean.shape}")
print(f"Cleaned test shape: {test_clean.shape}")

Cleaned training shape: (732, 14)
Cleaned test shape: (184, 13)


### Dataset Overview
This dataset contains heart disease diagnostic measurements from patients. The target variable `label` indicates disease severity (0 = no disease, 1-4 = increasing severity levels). 

**Key observations:**
- 732 training samples and 184 test samples
- 13 predictor features plus 1 target variable
- Mix of numerical and categorical features
- Several features have mixed data types (object) indicating potential encoding issues

---
## 2. Missing Values Analysis

In [27]:
missing_train = (train_clean.isnull().sum() / len(train_clean) * 100)
missing_train = missing_train[missing_train > 0].sort_values(ascending=False)

print(f"Features with missing values: {len(missing_train)}")
print(f"Total missing: {train_clean.isnull().sum().sum()} cells")
print(f"\nMissing percentages:\n{missing_train}")

plots.plot_missing_values(
    missing_train,
    title="Missing Values in Training Set",
    threshold=20,
    save_path="../images/missing_values.png"
)

Features with missing values: 9
Total missing: 1376 cells

Missing percentages:
ca          65.437158
thal        51.775956
slope       33.743169
fbs          9.016393
oldpeak      6.693989
trestbps     6.420765
thalach      6.010929
exang        6.010929
chol         2.868852
dtype: float64


### Understanding Missing Data Patterns

This dataset uses multiple encoding schemes for missing values:
- `?` - Standard missing value indicator
- `-9` - Another missing value encoding (likely from data source)
- `0` in some features (e.g., cholesterol) - Physiologically impossible values that represent missing data

**Critical Features with Missingness:**
- **ca (number of major vessels)**: ~60% missing - this is a fluoroscopy measurement that may not be performed on all patients
- **thal (thalassemia)**: ~50% missing - a blood disorder test not routinely performed
- **slope**: High missingness - ST slope from ECG, may be difficult to measure in some cases
- **chol (cholesterol)**: Contains zeros which are physiologically impossible, indicating missing data

**Implications for Analysis:**
The high missingness in `ca` and `thal` suggests these are expensive or invasive tests not performed on all patients. This will require careful imputation strategy or we may need to build models that work without these features.

---
## 3. Target Variable Analysis

In [5]:
target_counts = train_clean['label'].value_counts().sort_index()
target_pct = (target_counts / len(train_clean) * 100).round(2)

for label, count in target_counts.items():
    print(f"Label {label}: {count:3d} samples ({target_pct[label]:5.2f}%)")

plots.plot_target_distribution(
    train_clean['label'],
    title="Heart Disease Severity Distribution"
)

Label 0: 327 samples (44.67%)
Label 1: 156 samples (21.31%)
Label 2: 108 samples (14.75%)
Label 3: 107 samples (14.62%)
Label 4:  34 samples ( 4.64%)


### Target Distribution Insights

The target variable shows the severity of heart disease:
- **Label 0**: No disease (healthy)
- **Labels 1-4**: Increasing disease severity

**Key Findings:**
- Class imbalance exists, with some severity levels underrepresented
- This imbalance may require stratified sampling or class weighting in models
- Consider binary classification (disease vs. no disease) vs. multi-class severity prediction

---
## 4. Numerical Features Analysis

In [6]:
numerical_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

display(train_clean[numerical_cols].describe())

plots.plot_numerical_distributions(
    train_clean,
    cols=numerical_cols,
    bins=30
)

,age,trestbps,chol,thalach,oldpeak
count,732.000000,685.000000,711.000000,688.000000,683.000000
mean,53.364754,131.975182,199.956399,138.132267,0.881259
std,9.306868,19.203305,110.928639,25.963443,1.112960
min,28.000000,0.000000,0.000000,60.000000,-2.600000
25%,47.000000,120.000000,176.500000,120.000000,0.000000
50%,54.000000,130.000000,223.000000,140.000000,0.500000
75%,60.000000,140.000000,268.500000,158.250000,1.550000
max,76.000000,200.000000,603.000000,202.000000,6.200000


In [7]:
for col in numerical_cols:
    print(f"\n{col}:")
    print(train_clean.groupby('label')[col].describe()[['mean', 'std', '50%']])
    plots.plot_distributions_by_label(train_clean, col, label_col='label')


age:
            mean       std   50%
label                           
0      50.461774  9.461461  51.0
1      54.320513  9.058225  56.0
2      56.314815  7.647183  57.0
3      57.532710  8.181108  58.0
4      54.411765  8.403166  54.5

trestbps:
             mean        std    50%
label                              
0      130.376206  17.206618  130.0
1      128.891892  19.660775  125.0
2      135.290000  16.794236  134.0
3      136.557895  24.540768  135.0
4      138.000000  20.557237  136.0

chol:
             mean         std    50%
label                               
0      229.552716   75.174698  228.0
1      168.225806  120.149990  216.0
2      175.257143  138.277037  218.0
3      177.542857  129.124942  220.0
4      218.181818  121.164262  231.0

thalach:
             mean        std    50%
label                              
0      149.607717  23.459240  152.0
1      134.601351  23.164816  132.5
2      128.019802  21.729694  129.0
3      121.718750  25.986111  120.0
4      1

### Feature Definitions and Clinical Significance

**Cardiovascular Measurements:**
- **age**: Patient age in years - risk increases with age
- **trestbps**: Resting blood pressure (mm Hg) - hypertension is a major risk factor
- **chol**: Serum cholesterol (mg/dl) - high cholesterol indicates increased risk
- **thalach**: Maximum heart rate achieved during stress test - lower values may indicate cardiac issues
- **oldpeak**: ST depression induced by exercise (ECG measurement) - higher values suggest ischemia

**Key Distribution Patterns:**
- **Age**: Right-skewed distribution, most patients 50-65 years old
- **thalach**: Left-skewed, most patients achieve 140-170 bpm
- **oldpeak**: Right-skewed with many zero values (no ST depression observed)
- **trestbps & chol**: Roughly normal distributions centered around healthy ranges

**Clinical Interpretation:**
The distributions align with medical expectations - older patients and those with abnormal cardiac responses (low max heart rate, high ST depression) are more likely in a heart disease study.

---
## 5. Categorical Features Analysis

In [8]:
categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

for col in categorical_cols:
    if col in train_clean.columns:
        print(f"\n{col}:")
        print(train_clean[col].value_counts().sort_index())

plots.plot_categorical_distributions(
    train_clean,
    cols=categorical_cols
)


sex:
sex
0.0    154
1.0    578
Name: count, dtype: int64

cp:
cp
1.0     35
2.0    138
3.0    168
4.0    391
Name: count, dtype: int64

fbs:
fbs
0.0    554
1.0    112
Name: count, dtype: int64

restecg:
restecg
0.0    439
1.0    145
2.0    148
Name: count, dtype: int64

exang:
exang
0.0    422
1.0    266
Name: count, dtype: int64

slope:
slope
1.0    158
2.0    277
3.0     50
Name: count, dtype: int64

ca:
ca
0.0    147
1.0     54
2.0     35
3.0     17
Name: count, dtype: int64

thal:
thal
3.0    157
6.0     36
7.0    160
Name: count, dtype: int64


In [9]:
for col in categorical_cols:
    if col in train_clean.columns:
        print(f"\n{col}:")
        ct = pd.crosstab(train_clean[col], train_clean['label'], margins=True)
        display(ct)
        plots.plot_distributions_by_label(train_clean, col, label_col='label')


sex:


label,0,1,2,3,4,All
sex,,,,,,
0.0,115,19,9,8,3,154
1.0,212,137,99,99,31,578
All,327,156,108,107,34,732



cp:


label,0,1,2,3,4,All
cp,,,,,,
1.0,21,5,5,3,1,35
2.0,117,14,2,5,0,138
3.0,102,27,15,19,5,168
4.0,87,110,86,80,28,391
All,327,156,108,107,34,732



fbs:


...

label,0,1,2,3,4,All
slope,,,,,,
1.0,100,30,13,13,2,158
2.0,64,74,60,61,18,277
3.0,10,11,12,10,7,50
All,174,115,85,84,27,485



ca:


label,0,1,2,3,4,All
ca,,,,,,
0.0,109,22,7,6,3,147
1.0,18,14,13,8,1,54
2.0,8,6,8,13,0,35
3.0,2,3,4,5,3,17
All,137,45,32,32,7,253



thal:


label,0,1,2,3,4,All
thal,,,,,,
3.0,112,23,12,10,0,157
6.0,8,9,10,6,3,36
7.0,32,47,34,39,8,160
All,152,79,56,55,11,353


### Categorical Feature Definitions

**Patient Characteristics:**
- **sex**: 1 = male, 0 = female (males have higher heart disease prevalence)
- **fbs**: Fasting blood sugar > 120 mg/dl (1 = true, diabetes indicator)
- **exang**: Exercise-induced angina (1 = yes, indicates cardiac stress issues)

**Cardiac Test Results:**
- **cp (chest pain type)**: 
  - 1 = typical angina
  - 2 = atypical angina  
  - 3 = non-anginal pain
  - 4 = asymptomatic
- **restecg (resting ECG results)**:
  - 0 = normal
  - 1 = ST-T wave abnormality
  - 2 = left ventricular hypertrophy
- **slope**: ST segment slope during peak exercise
  - 1 = upsloping (normal)
  - 2 = flat (concerning)
  - 3 = downsloping (concerning)
- **ca**: Number of major vessels colored by fluoroscopy (0-3, more = worse)
- **thal (thalassemia)**: 
  - 3 = normal
  - 6 = fixed defect
  - 7 = reversible defect

**Key Observations:**
- Dataset is male-dominated (~79%), reflecting historical cardiovascular study demographics
- Chest pain type shows strong variation across disease severity
- Many patients have abnormal ECG results even at rest

---
## 6. Correlation Analysis

In [10]:
correlations = train_clean.corr()['label'].drop('label').sort_values(ascending=False)
print("\nFeatures most correlated with target:")
print(correlations)

plots.plot_correlation_matrix(
    train_clean,
    method='pearson'
)


Features most correlated with target:
ca          0.501912
oldpeak     0.482148
thal        0.468602
exang       0.427005
cp          0.397885
slope       0.351605
sex         0.270454
age         0.264917
trestbps    0.138043
fbs         0.134370
restecg     0.049529
chol       -0.149165
thalach    -0.407338
Name: label, dtype: float64


### Correlation Insights

**Strong Positive Correlations:**
- **slope & oldpeak** (0.58): Both measure exercise-induced cardiac stress, expected correlation
- **cp & exang** (moderate): Chest pain types relate to exercise-induced angina

**Strong Negative Correlations:**
- **thalach & age** (-0.39): Maximum heart rate decreases with age (physiologically expected)
- **exang & thalach** (-0.38): Exercise-induced angina associated with lower max heart rate
- **oldpeak & thalach** (-0.35): ST depression correlates with reduced cardiac capacity

**Target Correlations:**
Features most correlated with disease severity:
1. **cp (chest pain type)**: Different pain patterns indicate disease presence
2. **thalach**: Lower maximum heart rate suggests cardiac dysfunction
3. **oldpeak**: Exercise-induced ST depression is a key diagnostic indicator
4. **exang**: Exercise-induced angina directly indicates cardiac stress
5. **ca**: Number of blocked vessels directly measures disease severity

**Clinical Validation:**
These correlations align with medical knowledge - the strongest predictors are direct measurements of cardiac function under stress (exercise tests) and arterial blockage (fluoroscopy).

---
## 7. Outlier Detection

In [11]:
plots.plot_boxplots(train_clean, cols=numerical_cols)

for col in numerical_cols:
    Q1 = train_clean[col].quantile(0.25)
    Q3 = train_clean[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = train_clean[
        (train_clean[col] < lower_bound) | (train_clean[col] > upper_bound)
    ][col]
    
    print(f"\n{col}:")
    print(f"  Lower bound: {lower_bound:.2f}")
    print(f"  Upper bound: {upper_bound:.2f}")
    print(f"  Outliers: {len(outliers)} ({len(outliers)/len(train_clean)*100:.2f}%)")


age:
  Lower bound: 27.50
  Upper bound: 79.50
  Outliers: 0 (0.00%)

trestbps:
  Lower bound: 90.00
  Upper bound: 170.00
  Outliers: 24 (3.28%)

chol:
  Lower bound: 38.50
  Upper bound: 406.50
  Outliers: 146 (19.95%)

thalach:
  Lower bound: 62.62
  Upper bound: 215.62
  Outliers: 1 (0.14%)

oldpeak:
  Lower bound: -2.33
  Upper bound: 3.88
  Outliers: 13 (1.78%)


### Outlier Analysis and Handling

**Identified Outliers:**
Using IQR method (values beyond Q1 - 1.5×IQR or Q3 + 1.5×IQR)

**Decision on Outliers:**
- **Retain outliers**: These represent real physiological extremes (very high cholesterol, extreme ST depression)
- Removing them would discard potentially valuable information about severe cases
- Outliers may be important for predicting severe disease states
- Models like tree-based methods are robust to outliers

**Note:** If using distance-based models (SVM, KNN), consider robust scaling methods that reduce outlier impact.

---
## 8. Statistical Tests

### Chi-Square Tests (Categorical Features)

In [12]:
print("H0: Feature is independent of disease severity")
print("H1: Feature is associated with disease severity")
print("Significance level: α = 0.05\n")

chi_square_results = []

for col in categorical_cols:
    if col in train_clean.columns:
        contingency_table = pd.crosstab(
            train_clean[col].fillna(-1),
            train_clean['label']
        )
        
        chi2, p_value, dof, expected = chi2_contingency(contingency_table)
        
        chi_square_results.append({
            'Feature': col,
            'Chi-Square': chi2,
            'p-value': p_value,
            'DoF': dof,
            'Significant': 'Yes' if p_value < 0.05 else 'No'
        })
        
        print(f"{col:12s}: χ² = {chi2:8.2f}, p-value = {p_value:.4f}, DoF = {dof:2d} → "
              f"{'SIGNIFICANT' if p_value < 0.05 else 'Not significant'}")

chi_df = pd.DataFrame(chi_square_results).sort_values('p-value')
display(chi_df)

plots.plot_statistical_tests(chi_df, test_type='chi-square')

H0: Feature is independent of disease severity
H1: Feature is associated with disease severity
Significance level: α = 0.05

sex         : χ² =    72.06, p-value = 0.0000, DoF =  4 → SIGNIFICANT
cp          : χ² =   194.79, p-value = 0.0000, DoF = 12 → SIGNIFICANT
fbs         : χ² =    51.45, p-value = 0.0000, DoF =  8 → SIGNIFICANT
restecg     : χ² =    14.87, p-value = 0.0618, DoF =  8 → Not significant
exang       : χ² =   150.25, p-value = 0.0000, DoF =  8 → SIGNIFICANT
slope       : χ² =   130.21, p-value = 0.0000, DoF = 12 → SIGNIFICANT
ca          : χ² =    99.24, p-value = 0.0000, DoF = 16 → SIGNIFICANT
thal        : χ² =   102.06, p-value = 0.0000, DoF = 12 → SIGNIFICANT


,Feature,Chi-Square,p-value,DoF,Significant
1,cp,194.792641,3.867997e-35,12,Yes
4,exang,150.253918,1.735670e-28,8,Yes
5,slope,130.209627,5.602045e-22,12,Yes
7,thal,102.055878,2.200469e-16,12,Yes
0,sex,72.064375,8.317623e-15,4,Yes
6,ca,99.240348,4.806911e-14,16,Yes
2,fbs,51.448299,2.150652e-08,8,Yes
3,restecg,14.868078,6.176194e-02,8,No


### Chi-Square Test Results Interpretation

**Test Purpose**: Determine if categorical features are independent of disease severity

**Highly Significant Associations (p < 0.001):**
- **cp (chest pain type)**: Different pain patterns strongly associated with disease presence
  - Typical angina more common in disease cases
  - Asymptomatic cases more common in healthy patients
- **thal (thalassemia)**: Reversible defects strongly indicate disease
- **ca (vessel blockage)**: Direct measure of disease severity
- **exang (exercise-induced angina)**: Presence strongly indicates disease

**Moderate Associations:**
- **slope**: Exercise ST slope patterns differ by disease severity
- **sex**: Males have higher disease prevalence

**Weak/Non-Significant:**
- **fbs (fasting blood sugar)**: Diabetes indicator doesn't vary much by severity in this dataset
- **restecg**: Resting ECG less discriminative than exercise-based tests

**Key Insight:**
Exercise-induced symptoms (exang, cp, slope) and stress test measurements are far more predictive than resting measurements. This explains why exercise stress tests are gold standard in cardiology.

### ANOVA Tests (Numerical Features)

In [13]:
print("H0: Mean values are equal across all disease severity levels")
print("H1: At least one mean is different")
print("Significance level: α = 0.05\n")

anova_results = []

for col in numerical_cols:
    if col in train_clean.columns:
        groups = [
            train_clean[train_clean['label'] == label][col].dropna()
            for label in sorted(train_clean['label'].unique())
        ]
        
        f_stat, p_value = stats.f_oneway(*groups)
        
        anova_results.append({
            'Feature': col,
            'F-statistic': f_stat,
            'p-value': p_value,
            'Significant': 'Yes' if p_value < 0.05 else 'No'
        })
        
        print(f"{col:12s}: F = {f_stat:8.2f}, p-value = {p_value:.4f} → "
              f"{'SIGNIFICANT' if p_value < 0.05 else 'Not significant'}")

anova_df = pd.DataFrame(anova_results).sort_values('p-value')
display(anova_df)

plots.plot_statistical_tests(anova_df, test_type='anova')

H0: Mean values are equal across all disease severity levels
H1: At least one mean is different
Significance level: α = 0.05

age         : F =    18.10, p-value = 0.0000 → SIGNIFICANT
trestbps    : F =     4.44, p-value = 0.0015 → SIGNIFICANT
chol        : F =    12.04, p-value = 0.0000 → SIGNIFICANT
thalach     : F =    38.55, p-value = 0.0000 → SIGNIFICANT
oldpeak     : F =    51.95, p-value = 0.0000 → SIGNIFICANT


,Feature,F-statistic,p-value,Significant
4,oldpeak,51.948097,3.525834e-38,Yes
3,thalach,38.549848,4.112392e-29,Yes
0,age,18.098793,3.502627e-14,Yes
2,chol,12.041805,1.794259e-09,Yes
1,trestbps,4.440820,1.503312e-03,Yes


### ANOVA Test Results Interpretation

**Null Hypothesis (H0)**: Mean values are equal across all disease severity levels
**Alternative Hypothesis (H1)**: At least one group mean is significantly different
**Significance Level**: α = 0.05

**Highly Significant Features (p < 0.001):**
- **oldpeak**: F-statistic is very high, indicating strong discrimination between severity levels. ST depression increases with disease severity.
- **thalach**: Maximum heart rate shows clear separation between groups. Patients with disease achieve lower maximum heart rates.
- **ca**: Number of blocked vessels directly measures disease severity (when available).

**Moderately Significant Features (p < 0.05):**
- **age**: Older patients tend to have more severe disease
- **trestbps**: Blood pressure shows some variation across groups but weaker than other features

**Non-Significant Features:**
- **chol**: Surprisingly, cholesterol levels don't vary significantly by severity group in this dataset

**Implications for Modeling:**
1. Exercise test results (thalach, oldpeak) are the strongest predictors
2. These features should be prioritized in feature engineering
3. Age and blood pressure provide supplementary information
4. Cholesterol may be less important than expected, possibly due to medication use in the patient population